In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)

In [ ]:
train_df = pd.read_parquet("train.parquet")
test_df = pd.read_parquet("test.parquet")

In [ ]:
enc = {'not_spam': 0, 'spam': 1}
train_df['label'] = train_df['label'].map(enc)
test_df['label'] = test_df['label'].map(enc)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_texts(texts, labels):
    tokens = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    tokens["labels"] = torch.tensor(labels)
    return tokens

In [ ]:
class SpamDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []
best_f1 = 0.0
best_fold = 0

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kfold.split(train_df['text'], train_df['label'])):
    print(f"\n=== Fold {fold+1}/5 ===")

    train_texts = train_df.iloc[train_idx]['text'].values
    train_labels = train_df.iloc[train_idx]['label'].values
    val_texts = train_df.iloc[val_idx]['text'].values
    val_labels = train_df.iloc[val_idx]['label'].values

    train_encodings = tokenize_texts(train_texts, train_labels)
    val_encodings = tokenize_texts(val_texts, val_labels)

    train_dataset = SpamDataset(train_encodings)
    val_dataset = SpamDataset(val_encodings)

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold}",
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        learning_rate=1e-5,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        logging_dir=f"./logs_fold_{fold}",
        logging_steps=50,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()
    results.append(eval_results)

    # Save best model based on F1
    if eval_results["eval_f1"] > best_f1:
        best_f1 = eval_results["eval_f1"]
        best_fold = fold
        model.save_pretrained(f"./best_model")
        tokenizer.save_pretrained(f"./best_model")

# ========== Step 6: Evaluate Best Model on Test Set ==========
print(f"\nBest model found at fold {best_fold+1} with F1 = {best_f1:.4f}")
print("Evaluating on test set...")



=== Fold 1/5 ===


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.006000,0.003803,0.998777,1.000000,0.997576,0.998786
2,0.000300,0.000185,1.000000,1.000000,1.000000,1.000000
3,0.000100,0.001201,0.999388,1.000000,0.998788,0.999394
4,0.000100,0.000121,1.000000,1.000000,1.000000,1.000000
5,0.000100,0.000100,1.000000,1.000000,1.000000,1.000000


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.010400,0.010306,0.998165,0.997579,0.998788,0.998183
2,0.008900,0.005890,0.998777,0.998788,0.998788,0.998788
3,0.003400,0.008558,0.998165,0.996377,1.000000,0.998185
4,0.000100,0.005214,0.999388,0.998789,1.000000,0.999394
5,0.000100,0.006557,0.998165,0.998786,0.997576,0.998181



=== Fold 5/5 ===


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.008800,0.003482,0.998777,1.000000,0.997576,0.998786
2,0.009600,0.002468,0.999388,1.000000,0.998788,0.999394
3,0.001100,0.003781,0.999388,1.000000,0.998788,0.999394
4,0.000100,0.004232,0.999388,1.000000,0.998788,0.999394
5,0.000100,0.004308,0.999388,1.000000,0.998788,0.999394



Best model found at fold 1 with F1 = 1.0000
Evaluating on test set...


In [ ]:
test_encodings = tokenize_texts(test_df['text'].values, test_df['label'].values)
test_dataset = SpamDataset(test_encodings)

best_model = BertForSequenceClassification.from_pretrained("./best_model")
trainer = Trainer(
    model=best_model,
    compute_metrics=compute_metrics,
    args=TrainingArguments(
        output_dir="./no_log",
        report_to="none",
    )
)

test_results = trainer.evaluate(test_dataset)
print("Test set results:")
print(test_results)

Test set results:
{'eval_loss': 0.010932652279734612, 'eval_model_preparation_time': 0.0027, 'eval_accuracy': 0.9977981651376147, 'eval_precision': 0.9985433357611071, 'eval_recall': 0.9970909090909091, 'eval_f1': 0.9978165938864629, 'eval_runtime': 20.3679, 'eval_samples_per_second': 133.789, 'eval_steps_per_second': 16.742}


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=1)
y_true = test_df['label'].values

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

[[1348    2]
 [   4 1371]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1350
           1       1.00      1.00      1.00      1375

    accuracy                           1.00      2725
   macro avg       1.00      1.00      1.00      2725
weighted avg       1.00      1.00      1.00      2725

